In [1]:
import pybullet as p
import time
import numpy as np
import os
import _utils as u

In [2]:
p.connect(p.GUI)
p.createCollisionShape(p.GEOM_PLANE, planeNormal = [0,0,1])
p.createMultiBody(0,0)

0

In [3]:
sh_body       = p.createCollisionShape(p.GEOM_BOX,halfExtents=[0.1, 0.1, 0.1])
sh_extraweight= p.createCollisionShape(p.GEOM_BOX,halfExtents=[1.0, 1.0, 0.1])
paralelepiped = p.createCollisionShape(p.GEOM_BOX,halfExtents=[0.1, 0.2, 0.3])
qube          = p.createCollisionShape(p.GEOM_BOX,halfExtents=[0.1, 0.1, 0.1])
link          = p.createCollisionShape(p.GEOM_BOX,halfExtents=[0.01, 0.01, 0.01])
sphere        = p.createCollisionShape(p.GEOM_SPHERE,radius = 0.1)
CAPSULE       = p.createCollisionShape(p.GEOM_CAPSULE, height = 0.3, radius = 0.1)

In [4]:
def diag(l):
    return (l**2+l**2)**0.5

In [5]:
linkCollisionShapeIndices=[
    paralelepiped,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    sh_extraweight,
]
link_Masses=[
    1,
    1,1,1,1,10,
    1,1,1,1,20,
    
    1,1,1,1,1,
    1,1,1,1,1,
    1,1,1,1,1,
    1,1,1,1,1,
    2000
]
#link positions wrt the link they are attached to
linkPositions=[
    [0,0,2],
    [0,-0.3,0],[0,0,0],[0, 0,-0.3],[0,0,-0.3],[0,0,-0.3],
    [0, 0.3,0],[0,0,0],[0, 0,-0.3],[0,0,-0.3],[0,0,-0.3],
    
    [ diag(1), diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #front left
    [ diag(1),-diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #front right
    [-diag(1), diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #back left
    [-diag(1),-diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #back right
    [0,0,0.000001],
]
#indices determine for each link which other link it is attached to
# for example 3rd index = 2 means that the front left knee jjoint is attached to the front left hip
indices=[
    0,
    1,2,3,4, 5,
    1,7,8,9,10,
    
    0,12,13,14,15,
    0,17,18,19,20,
    0,22,23,24,25,
    0,27,28,29,30,
    0
]
#Most joint are revolving. The prismatic joints are kept fixed for now
jointTypes=[
    p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_FIXED,
]
#revolution axis for each revolving joint
axis=[
    [0,0,0],                                 #  0
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],#  1, 2, 3, 4, 5,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],#  6, 7, 8, 9,10,
    
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 11,12,13,14,15,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 16,17,18,19,20,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 21,22,23,24,25,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 26,27,28,29,30,
    
    [0,0,0],
]
legs_hip_f = [11,16,21,26]
legs_hip_s = [12,17,22,27]
knee       = [14,19,24,29] 

In [6]:
body_Mass                     = 2000
visualShapeId                 = -1
nlnk                          = len(link_Masses)
linkVisualShapeIndices        = [-1]*nlnk    #=[-1,-1,-1, ... , -1]
linkOrientations              = [[0,0,0,1]]*nlnk
linkInertialFramePositions    = [  [0,0,0]]*nlnk

#Note the orientations are given in quaternions (4 params). 
#There are function to convert of Euler angles and back
linkInertialFrameOrientations = [[0,0,0,1]]*nlnk

#Drop the body in the scene at the following body coordinates
basePosition                  = [0,0,2]
baseOrientation               = [0,0,0,1]

In [7]:
#Main function that creates the dog
dog = p.createMultiBody(body_Mass,
                        sh_body,
                        visualShapeId,
                        basePosition,
                        baseOrientation,
                        linkMasses=link_Masses,
                        linkCollisionShapeIndices=linkCollisionShapeIndices,
                        linkVisualShapeIndices=linkVisualShapeIndices,
                        linkPositions=linkPositions,
                        linkOrientations=linkOrientations,
                        linkInertialFramePositions=linkInertialFramePositions,
                        linkInertialFrameOrientations=linkInertialFrameOrientations,
                        linkParentIndices=indices,
                        linkJointTypes=jointTypes,
                        linkJointAxis=axis)

In [8]:
arms       = [1,6,2,7,4,9] 
legs_hip_f = [11,16,21,26]
legs_hip_s = [12,17,22,27]
knee       = [14,19,24,29] 
all_motors = arms+legs_hip_f+legs_hip_s+knee
linkValues = len(all_motors) * [0]
u.resetCoordinates(dog,all_motors,linkValues)

In [9]:
#Add earth like gravity
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(1)
u.resetCamera()

In [10]:
rightF = 1
leftF  = 6
rightS = 2
leftS  = 7
rightE = 4
leftE  = 9

In [11]:
def reset_position(body, index, mode, degrees,force,maxVelocity):
    p.setJointMotorControl2(
            body,
            index,
            mode,
            targetPosition = u.rad2deg(degrees),
            force          = force,
            maxVelocity    = maxVelocity
        )
def _SetMotorTorqueById(minitaur, motor_id, torque):
    p.setJointMotorControl2(bodyIndex      = minitaur,
                            jointIndex     = motor_id,
                            controlMode    = p.TORQUE_CONTROL,
                            force          = torque)

def _SetMotorAngleById(minitaur, motor_id, desired_angle#, force
                      ):
    p.setJointMotorControl2(bodyIndex      = minitaur,
                            jointIndex     = motor_id,
                            controlMode    = p.POSITION_CONTROL,
                            targetPosition = u.rad2deg(desired_angle),
                            #force          = force
                           )

In [15]:
body        = dog
mode        = p.POSITION_CONTROL
degrees     = 0
force       = 100
maxVelocity = 20
torque      = [3,3]
_SetMotorTorqueById(body, rightF, torque[0])
_SetMotorTorqueById(body, leftF,  torque[1])
_SetMotorAngleById(body, rightF, degrees)#,force)
_SetMotorAngleById(body, leftF,  degrees)#,force)

In [16]:
# КРУТЯЩИЙ МОМЕНТ – Сила, направленная по кругу (вращение объекта), называется крутящим моментом. 
# Крутящий момент - это вращающая сила. Если к объекту приложен крутящий момент, на границе первого 
# возникает линейная сила. В примере с колесом, катящемся по земле, крутящий момент, приложенный к 
# оси колеса, создает линейную силу на границе покрышки в точке ее контакта с поверхностью земли. 
# Так и определяется крутящий момент - как линейная сила на границе круга. Крутящий момент определяется 
# величиной силы, умноженной на расстояние от центра вращения (Сила х Расстояние = Крутящий момент). 
# Крутящий момент измеряется в единицах силы, умноженной на расстояние, например, фунто-дюймах или 
# ньютон-метрах.
# https://www.programcreek.com/python/?code=alexsax%2Fmidlevel-reps%2Fmidlevel-reps-master%2Fgibson%2Fgibson%2Fcore%2Fphysics%2Fdrivers%2Fminitaur.py#

In [17]:
body        = dog
index       = rightF 
mode        = p.POSITION_CONTROL
degrees     = 180
force       = 20


_SetMotorAngleById(body, rightF, degrees)#,force)
_SetMotorAngleById(body, leftF,  degrees)#,force)

In [ ]:
p.applyExternalTorque(
    objectUniqueId  = body,     # (int)object unique id as returned by load methods.
    linkIndex       = jointIdx, # (int)link index or -1 for the base.
    forceObj        =           # (vec3, list of 3 floats) force/torque vector to be applied [x,y,z]. 
                                #        See flags for coordinate system.
    posObj          =           # (vec3, list of 3 floats) position on the link where the force is applied. 
                                #        Only for applyExternalForce. See flags for coordinate system.
    flags           =           # (int) Specify the coordinate system of force/position: 
                                #       either WORLD_FRAME for Cartesian world coordinates 
                                #       or LINK_FRAME for local link coordinates.
    physicsClientId = -1
)

In [ ]:
from collections import namedtuple
ROBOT = namedtuple('Body',
                   ['linkCollisionShapeIndices',
                    'link_Masses',
                    'linkPositions',
                    'indices',
                    'jointTypes',
                    'axis'])

In [ ]:
linkCollisionShapeIndices=[
    paralelepiped,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    link,link,CAPSULE,link,CAPSULE,
    sh_extraweight,
]
link_Masses=[
    1,
    1,1,1,1,1,
    1,1,1,1,1,
    
    1,1,1,1,1,
    1,1,1,1,1,
    1,1,1,1,1,
    1,1,1,1,1,
    2000
]
#link positions wrt the link they are attached to
linkPositions=[
    [0,0,2],
    [0,-0.3,0],[0,0,0],[0,-0.3,-0.3],[0,0,-0.3],[0,0,-0.3],
    [0, 0.3,0],[0,0,0],[0, 0.3,-0.3],[0,0,-0.3],[0,0,-0.3],
    
    [ diag(1), diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #front left
    [ diag(1),-diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #front right
    [-diag(1), diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #back left
    [-diag(1),-diag(1),0],[0,0,0],[0,0,-0.3],[0,0,-0.3],[0,0,-0.3], #back right
    [0,0,0.000001],
]
#indices determine for each link which other link it is attached to
# for example 3rd index = 2 means that the front left knee jjoint is attached to the front left hip
indices=[
    0,
    1,2,3,4, 5,
    1,7,8,9,10,
    
    0,12,13,14,15,
    0,17,18,19,20,
    0,22,23,24,25,
    0,27,28,29,30,
    0
]
#Most joint are revolving. The prismatic joints are kept fixed for now
jointTypes=[
    p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_REVOLUTE, p.JOINT_REVOLUTE,  p.JOINT_PRISMATIC,p.JOINT_REVOLUTE, p.JOINT_PRISMATIC,
    p.JOINT_FIXED,
]
#revolution axis for each revolving joint
axis=[
    [0,0,0],                                 #  0
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],#  1, 2, 3, 4, 5,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],#  6, 7, 8, 9,10,
    
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 11,12,13,14,15,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 16,17,18,19,20,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 21,22,23,24,25,
    [0,1,0],[1,0,0],[0,0,0],[0,-1,0],[0,0,0],# 26,27,28,29,30,
    
    [0,0,0],
]

In [ ]:
indices=[
    0, # corpus
    1, # j_r_shoulder
    2, # j_r_shoulder
    3, # m_r_shoulder
    4, # j_r_elbow
    5, # m_r_elbow
    
    1, # j_l_shoulder
    7, # j_l_shoulder
    8, # m_l_shoulder
    9, # j_l_elbow
    10,# m_l_elbow
    
    0, # j_fl_shoulder
    12,# j_fl_shoulder
    13,# j_fl_shoulder
    14,# j_fl_shoulder
    15,# j_fl_shoulder
    
    0, # j_fr_shoulder
    17,# j_fr_shoulder
    18,# j_fr_shoulder
    19,# j_fr_shoulder
    20,# j_fr_shoulder
    
    0, # j_br_shoulder
    22,# j_br_shoulder
    23,# j_br_shoulder
    24,# j_br_shoulder
    25,# j_br_shoulder
    
    0, # j_br_shoulder
    27,# j_br_shoulder
    28,# j_br_shoulder
    29,# j_br_shoulder
    30,# j_br_shoulder
    0
]
p.setJointMotorControl2(
    multi_body,
    motor_index,
    p.POSITION_CONTROL,
    targetPosition = rad2deg(degrees),
    force          = force,
    maxVelocity    = maxVelocity
)